In [ ]:
import numpy as np
import pandas as pd
from scipy import integrate

%cd -q ../..
import utils.constants as const
from atoms.atomic_species import atomic_species
from stars.spectrum.lisard import lisard_spectrum

from stars.spectrum.high_energy_models.Linsky2014 import f_uv_bins

import matplotlib.pyplot as plt
%matplotlib inline

# Plot LISARD spectrums

## uv spectrums: FISM2, FISM, SEE, EVE, MAVEN, compared with Linsky et. al 2014

Will fetch each dataset from LISARD, may take a few minutes. You can comment out missions to skip downloading data and run faster. Data is saved as a parquet in a folder called McAstro/stars/spectrum/lisard_spectrums. 

In [ ]:
# Missions
date = '2016-01-01'
fism2 = lisard_spectrum(date=date, mission='fism2')
fism = lisard_spectrum(date=date, mission='fism')
see = lisard_spectrum(date=date, mission='see')
eve = lisard_spectrum(date=date, mission='eve')
maven = lisard_spectrum(date=date, mission='maven')
spectrums = [fism2, fism, see, eve, maven]

# Linsky
wl_bins = np.array([10, 20, 30, 40, 50, 60, 70, 80, 91.2, 117])*1e-7
wl = np.linspace(wl_bins[0], wl_bins[-1], 1024)
nu = const.c/wl
Linsky_bin = [i for i in range(len(wl_bins)-1)
              for w in wl if w >= wl_bins[i] and w <= wl_bins[i+1]]
Linsky_spectrum = f_uv_bins(5)
F_Linsky = Linsky_spectrum[Linsky_bin]
F_wl = F_Linsky/(wl_bins[[x+1 for x in Linsky_bin]]-wl_bins[Linsky_bin])
F_nu = F_Linsky/(const.c/wl_bins[Linsky_bin]
                 -const.c/wl_bins[[x+1 for x in Linsky_bin]])
linsky = lisard_spectrum(mission='linsky')
linsky.data = pd.DataFrame(np.array([wl, nu, F_wl, F_nu]).T,
                           columns=['wl', 'nu', 'F_wl', 'F_nu'])
spectrums += [linsky]

In [ ]:
fig, ax = plt.subplots()

for spectrum in spectrums:
    mask = (spectrum.data['wl']<=wl_bins[-1])&(spectrum.data['wl']>=wl_bins[0])
    masked_data = spectrum.data[mask]
    
    ax.plot(masked_data['wl']*1e7, masked_data['F_wl']*1e-7, lw=2,
            label=spectrum.mission)
    
ax.set_yscale('log')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'Irradiance (erg/s/cm$^2$/nm)')

fig.tight_layout()

## FISM2 solar min and max compared to Linsky

In [ ]:
# Wavelength range (hydrogen ionization edge)
absorber = atomic_species('H I')
ion_pot = absorber.verner_data['E_th']*const.eV
min_wl = 10e-7
max_wl = const.hc/ion_pot

#Missions
solar_minimum = '2009-01-01'
solar_maximum = '2002-01-01'
fism2_min = lisard_spectrum(date=solar_minimum, mission='fism2')
fism2_max = lisard_spectrum(date=solar_maximum, mission='fism2')
fism2_min.mission = 'fism2 (solar min)'
fism2_max.mission = 'fism2 (solar max)'
spectrums = [fism2_min, fism2_max]

# Linsky f_Lya set to match e_tot of fism2
wl_bins = np.array([10, 20, 30, 40, 50, 60, 70, 80, 91.2, 117])*1e-7
wl = np.linspace(min_wl, max_wl, 1024)
nu = const.c/wl
Linsky_bin = [i for i in range(len(wl_bins)-1)
              for w in wl if w >= wl_bins[i] and w <= wl_bins[i+1]]

Linsky_spectrum = f_uv_bins(4.8)
F_Linsky = Linsky_spectrum[Linsky_bin]
F_wl = F_Linsky/(wl_bins[[x+1 for x in Linsky_bin]]-wl_bins[Linsky_bin])
F_nu = F_Linsky/(const.c/wl_bins[Linsky_bin]
                 -const.c/wl_bins[[x+1 for x in Linsky_bin]])
linsky = lisard_spectrum(mission='linsky')
linsky.data = pd.DataFrame(np.array([wl, nu, F_wl, F_nu]).T,
                           columns=['wl', 'nu', 'F_wl', 'F_nu'])
spectrums += [linsky]
Linsky_spectrum = f_uv_bins(9.69)
F_Linsky = Linsky_spectrum[Linsky_bin]
F_wl = F_Linsky/(wl_bins[[x+1 for x in Linsky_bin]]-wl_bins[Linsky_bin])
F_nu = F_Linsky/(const.c/wl_bins[Linsky_bin]
                 -const.c/wl_bins[[x+1 for x in Linsky_bin]])
linsky = lisard_spectrum(mission='linsky')
linsky.data = pd.DataFrame(np.array([wl, nu, F_wl, F_nu]).T,
                           columns=['wl', 'nu', 'F_wl', 'F_nu'])
spectrums += [linsky]

In [ ]:
fig, ax = plt.subplots()

for spectrum in spectrums:
    mask = (spectrum.data['wl']<=max_wl)&(spectrum.data['wl']>=min_wl)
    masked_data = spectrum.data[mask]
    
    ax.plot(masked_data['wl']*1e7, masked_data['F_wl']*1e-7, lw=2,
            label=spectrum.mission)
    
ax.legend()
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'Irradiance (erg/s/cm$^2$/nm)')

ax.set_yscale('log')
fig.tight_layout()

# Calculate cycle averages

In [ ]:
import datetime

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)

In [ ]:
# Sol cycle 23
start_date = datetime.date(1996, 8, 1)
end_date = datetime.date(2008, 12, 1)
wl_0 = 10e-7
wl_1 = const.hc/(atomic_species('H I').verner_data['E_th']*const.eV)
ndays = (end_date - start_date).days
xuv = np.zeros(ndays)
dates = []

fism2 = lisard_spectrum(date='2000-01-01', mission='fism2',
                        sort_values='wl')
avg_spectrum = fism2.data
avg_spectrum['F_wl'] = np.zeros(len(a1.data['F_wl']))

for i, single_date in enumerate(daterange(start_date, end_date)):
    date = single_date.strftime("%Y-%m-%d")
    fism2 = lisard_spectrum(date=date, mission='fism2', sort_values='wl')
    ionz = fism2.data.loc[(fism2.data['wl']<=wl_1)&(fism2.data['wl']>=wl_0)]
    xuv[i] = integrate.simps(ionz['F_wl'], ionz['wl'])
    dates.append(single_date)
    avg_spectrum['F_wl'] += fism2.data['F_wl']/ndays
    if xuv[i] >= 6:
        print(f'{date}: {xuv[i]}')

In [ ]:
fig, ax = plt.subplots()
ax.plot(dates, xuv)
ax.axhline(xuv.mean(), ls='--', c='g', zorder=0)
ax.text(dates[-len(dates)//4], 0.1+xuv.mean(), f'mean: {xuv.mean():.2f}')

ax.axhline(xuv.min(), ls='--', c='r', zorder=0)
ax.text(dates[-2*len(dates)//3], 0.1+xuv.min(), f'min: {xuv.min():.2f}')

ax.axhline(xuv.max(), ls='--', c='b', zorder=0)
ax.text(dates[-len(dates)//4], 0.1+xuv.max(), f'max: {xuv.max():.2f}')
ax.set_ylim([1.5,5.5])
ax.set_title('Integrated xuv ($10-91.2$ nm) [cycle 23]')
ax.set_xlabel('Date')
ax.set_ylabel(r'xuv flux at Earth (erg/s/cm$^2$)')
fig.tight_layout()

In [ ]:
max_spectrum = lisard_spectrum(
    date=dates[np.where(xuv==xuv.max())[0][0]].strftime('%Y-%m-%d'),
    mission='fism2', sort_values='wl'
)
min_spectrum = lisard_spectrum(
    date=dates[np.where(xuv==xuv.min())[0][0]].strftime('%Y-%m-%d'),
    mission='fism2', sort_values='wl'
)

fig, ax = plt.subplots()
mk = (avg_spectrum['wl']>=wl_0)&(avg_spectrum['wl']<=wl_1)
ax.plot(avg_spectrum['wl'][mk]*1e7,
        avg_spectrum['F_wl'][mk], lw=1)
ax.plot(max_spectrum.data['wl'][mk]*1e7,
        max_spectrum.data['F_wl'][mk], lw=1)
ax.plot(min_spectrum.data['wl'][mk]*1e7,
        min_spectrum.data['F_wl'][mk], lw=1)
ax.set_yscale('log')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
mk = (avg_spectrum['wl']>=wl_0)&(avg_spectrum['wl']<=wl_1)
ax.plot(max_spectrum.data['wl'][mk]*1e7,
        max_spectrum.data['F_wl'][mk]/min_spectrum.data['F_wl'][mk], lw=1)
ax.plot(max_spectrum.data['wl'][mk]*1e7,
        max_spectrum.data['F_wl'][mk]/avg_spectrum['F_wl'][mk], lw=1)
ax.plot(min_spectrum.data['wl'][mk]*1e7,
        avg_spectrum['F_wl'][mk]/min_spectrum.data['F_wl'][mk], lw=1)
ax.axhline(xuv.max()/xuv.min(), zorder=0, ls='--', c='r')
ax.set_yscale('log')
ax.set_ylim([1,5])
fig.tight_layout()